In [8]:
import requests
import pandas as pd

def get_house_members():
    
    headers = {
        'X-API-Key': 'aLPU1BjfQphfOWjE0Z7Y1tB2pRRvGJl5loZe37U3',
        'Content-Type': 'application/json',
        'Accept': 'application/json'
       }
    
    response = requests.get('https://api.propublica.org/congress/v1/115/house/members.json', headers=headers)
    if response.status_code != 200:
        return False
    
    data = response.json()
    house = data['results'][0]['members']
    df = pd.DataFrame (house)

    df = df[['first_name', 'last_name','cspan_id', 'office', 'id', 'leadership_role' ,'date_of_birth', 'party', 'state', 'district', 'votes_with_party_pct', 'in_office' ,'twitter_account', 'title', 'facebook_account', 'phone', 'url']]
    return df

def get_senate_members():
    headers = {
        'X-API-Key': 'aLPU1BjfQphfOWjE0Z7Y1tB2pRRvGJl5loZe37U3',
        'Content-Type': 'application/json',
        'Accept': 'application/json'
       }
    
    response = requests.get('https://api.propublica.org/congress/v1/115/senate/members.json', headers=headers)
    if response.status_code != 200:
        return False
    
    data = response.json()
    house = data['results'][0]['members']
    df = pd.DataFrame (house)

    df = df[['first_name', 'last_name', 'id', 'office', 'cspan_id', 'leadership_role' ,'date_of_birth', 'party', 'state', 'votes_with_party_pct', 'in_office' ,'twitter_account', 'title', 'facebook_account', 'phone', 'url']]
    df['district'] = "N/A"
    return df

df_2 = get_house_members()
print(type(df_2.iloc[4]['url']))

<class 'str'>


In [5]:
import requests
import pandas as pd

def get_bills():
    
    headers = {
        'X-API-Key': 'aLPU1BjfQphfOWjE0Z7Y1tB2pRRvGJl5loZe37U3',
        'Content-Type': 'application/json',
        'Accept': 'application/json'
       }
    
    response = requests.get('https://api.propublica.org/congress/v1/bills/subjects/energy.json', headers=headers)
    if response.status_code != 200:
        return False
    
    data = response.json()
    bills = data['results']
    df = pd.DataFrame(bills)
    return df.loc[df['primary_subject'] == ""]
    #return df.primary_subject.unique()
    # Taxation
    # Public Lands and Natural Resources
    # Transportation and Public Works
    # Energy
    # Water Resources Development

print(get_bills())

Empty DataFrame
Columns: [active, bill_id, bill_slug, bill_type, bill_uri, committee_codes, committees, congressdotgov_url, cosponsors, cosponsors_by_party, enacted, govtrack_url, gpo_pdf_uri, house_passage, introduced_date, last_vote, latest_major_action, latest_major_action_date, number, primary_subject, senate_passage, short_title, sponsor_id, sponsor_name, sponsor_party, sponsor_state, sponsor_title, sponsor_uri, subcommittee_codes, summary, summary_short, title, vetoed]
Index: []

[0 rows x 33 columns]


In [6]:
import requests
import pandas as pd

def get_bills():
    
    headers = {
        'X-API-Key': 'aLPU1BjfQphfOWjE0Z7Y1tB2pRRvGJl5loZe37U3',
        'Content-Type': 'application/json',
        'Accept': 'application/json'
       }
    
    offset = 0
    
    
    bills = []
    while offset <= 60: #7200
        response = requests.get('https://api.propublica.org/congress/v1/bills/subjects/energy.json?offset={}'.format(offset), headers=headers)
        if response.status_code != 200:
            return False

        data = response.json()
#         bills.append(data['results'])
        for val in data['results']:
            bills.append(val)
        offset += 20
    
#     bills = data['results']
    df = pd.DataFrame(bills)
    
#     return df.loc[df['primary_subject'] == "Taxation"]
    return df

print(get_bills().iloc[15])
#print(get_bills().iloc[19])

active                                                                  False
bill_id                                                            hr2924-114
bill_slug                                                              hr2924
bill_type                                                                  hr
bill_uri                    https://api.propublica.org/congress/v1/114/bil...
committee_codes                                                        [HSII]
committees                                  House Natural Resources Committee
congressdotgov_url          https://www.congress.gov/bill/114th-congress/h...
cosponsors                                                                  0
cosponsors_by_party                                                        {}
enacted                                                                  None
govtrack_url                https://www.govtrack.us/congress/bills/114/hr2924
gpo_pdf_uri                                                     

In [7]:
def get_bills():

    headers = {
        'X-API-Key': 'aLPU1BjfQphfOWjE0Z7Y1tB2pRRvGJl5loZe37U3',
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }

    master_df = pd.DataFrame()
    more = True
    offset = 0	

    while (more):
        #print("x")
        response = requests.get('https://api.propublica.org/congress/v1/bills/subjects/energy.json?offset={}'.format(offset), headers=headers)
        if response.status_code != 200:
            #print("Here")
            return None

        data = response.json()
        bills = data['results']
        if (len(bills) == 0):
            more = False
            continue

        """
        flattens nested parts, ex.
        cosponsors_by_party: {
            "D": 1,
            "R": 2
        }
        becomes
        cosponsors_by_party_D: 1,
        cosponsors_by_party_R: 2

        need to do this to convert to df
        """
        df = pd.io.json.json_normalize(bills, sep='_')

        """
        can establish link to CongressMember record using sponsor_id (same as member_id returned
        by API call for members)
        """
        df = df[['bill_id', 'bill_type', 'title', 'short_title', 'sponsor_id',
                'introduced_date', 'cosponsors_by_party_D', 'committees',
                'active', 'last_vote', 'enacted', 'vetoed', 'congressdotgov_url', 'govtrack_url',
                'house_passage', 'senate_passage', 'summary', 'summary_short']]
        # TODO: need list of primary subjects that we chose
        #df = df[df.primary_subject in ['Energy']]
        # TODO: get cosponsors using https://api.propublica.org/congress/v1/{con_num}/bills/{bill-id}/cosponsors.json
        pd.concat([master_df, df], ignore_index=True)

        offset += 20

    return master_df

print(get_bills())

KeyboardInterrupt: 

In [10]:
def get_cosponsors(bill_id):
    headers = {
        'X-API-Key': 'aLPU1BjfQphfOWjE0Z7Y1tB2pRRvGJl5loZe37U3',
        'Content-Type': 'application/json',
        'Accept': 'application/json'
       }
    
    info = bill_id.split('-')
    bill_num = info[0]
    con_num = info[1]

    response = requests.get('https://api.propublica.org/congress/v1/{}/bills/{}/cosponsors.json'.format(con_num, bill_num), headers=headers)
    if (response.status_code != 200):
        return None

    cs_list = []

    data = response.json()['results'][0]['cosponsors']
    df = pd.DataFrame(data)
    return df['cosponsor_id'].tolist()

get_cosponsors('s715-109')

['D000596', 'D000563', 'L000123']

In [11]:
import requests

headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json'
   }

issues = {}
response = requests.get('https://gitlab.com/api/v4/projects/14474532/issues', headers=headers)
for val in response.json():
    print(val['iid'])

52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
